In [3]:
import pandas as pd

# Path to the downloaded file in Kaggle environment
file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
print("Before date conversion:")
print(data.head())

# Convert the 'Datetime' column to datetime
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Filter the data to keep only rows from the year 2010
data = data[data['Datetime'].dt.year >= 2010]

# Format the 'Datetime' column to the desired format: %Y-%m-%d %H:%M
data['Datetime'] = data['Datetime'].dt.strftime('%Y-%m-%d %H:%M')

# Display the first few rows after filtering and conversion
print("After filtering for >=2010 and date conversion:")
print(data.head())

# Save the modified DataFrame back to a new CSV file
output_file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv"
data.to_csv(output_file_path, index=False)

# Confirm that the file has been saved
print(f"Modified file saved as {output_file_path}")

Before date conversion:
              Datetime  ABI   ACT  AMA  ALB  JFK  LGA
0  1949-01-01 00:00:00  6.1  10.6  2.2 -3.3 -3.3 -3.9
1  1949-01-01 01:00:00  3.3   6.7  1.1 -3.3 -3.3 -3.9
2  1949-01-01 02:00:00  2.2   5.6  0.6 -3.3 -3.3 -3.9
3  1949-01-01 03:00:00  1.7   4.4  0.0 -2.8 -3.3 -3.9
4  1949-01-01 04:00:00  1.7   3.9  1.1 -2.8 -3.3 -3.9
After filtering for >=2010 and date conversion:
                Datetime  ABI  ACT  AMA  ALB  JFK  LGA
470398  2010-01-01 00:00  1.1  6.7 -0.6 -1.7  1.1  1.1
470399  2010-01-01 01:00  1.1  6.1 -3.3 -1.7  1.1  1.1
470400  2010-01-01 02:00  1.7  5.6 -3.9 -1.7  0.6  1.1
470401  2010-01-01 03:00  1.1  5.0 -3.9 -1.7  0.6  1.1
470402  2010-01-01 04:00  1.1  4.4 -5.0 -1.1  0.6  0.6
Modified file saved as S:\spatiotemporal-analysis\spacetimeformer-main\spacetimeformer\data\temperature-v1_modified.csv


!python S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\train.py spacetimeformer asos --context_points 128 --target_points 40 --run_name spacetimeformer_asos --batch_size 32 --warmup_steps 1000 --d_model 200 --d_ff 700 --enc_layers 1 --dec_layers 1 --dropout_emb .1 --dropout_ff .3 --d_qk 30 --d_v 30 --n_heads 10 --patience 10 --decay_factor .8 --data_path S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv